# docker-image-size-limit case

Recently, I have created a simple script to limit the size of `docker` images.
https://github.com/wemake-services/docker-image-size-limit

It was a distributable version of these four lines in `bash`:

```bash
LIMIT=1024
IMAGE='your-image-name:latest'

SIZE="$(docker image inspect "$IMAGE" --format='{{.Size}}')"
test "$SIZE" -gt "$LIMIT" && echo 'Limit exceeded'; false
```


## Python version

That's how my source code in `python` looks like at the moment:
https://github.com/wemake-services/docker-image-size-limit/blob/74835115ed80ff279b1004c8c2c8dff3161cd20e/docker_image_size_limit.py

```python
# -*- coding: utf-8 -*-

import argparse
import sys
from typing import NoReturn

import pkg_resources
from docker import DockerClient, from_env
from humanfriendly import format_size, parse_size

_version = pkg_resources.get_distribution(
    'docker_image_size_limit',
).version


def main() -> NoReturn:
    """Main CLI entrypoint."""
    client = from_env()
    arguments = _parse_args()
    oversize = check_image_size(client, arguments.image, arguments.size)

    exit_code = 0
    if oversize > 0:
        print('{0} exceeds {1} limit by {2}'.format(  # noqa: T001
            arguments.image,
            arguments.size,
            format_size(oversize, binary=True),
        ))
        exit_code = 1
    sys.exit(exit_code)


def check_image_size(client: DockerClient, image: str, limit: str) -> int:
    """
    Checks the image size of given image name.
    Compares it to the given size in bytes or in human readable format.
    Returns:
        Tresshold overflow in bytes. Or ``0`` if image is less than limit.
    """
    image_size = client.images.get(image).attrs['Size']

    try:
        image_limit = int(limit)
    except ValueError:
        image_limit = parse_size(limit, binary=True)

    if image_size > image_limit:
        return image_size - image_limit
    return 0


def _parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        description='Keep your docker images small',
    )
    parser.add_argument(
        '--version', action='version', version=_version,
    )
    parser.add_argument(
        'image', type=str, help='Docker image name to be checked',
    )
    parser.add_argument(
        'size', type=str, help='Human-readable size limit: 102 MB, 1GB',
    )
    return parser.parse_args()
```

It takes only ~70 lines of code, including declarative configuration, meta information, and docstrings.


## Hidden errors

Can you imagine how many lines where actually checked?
I have written 9 tests for this simple module: https://github.com/wemake-services/docker-image-size-limit/tree/74835115ed80ff279b1004c8c2c8dff3161cd20e/tests

Including:
- 4 E2E tests that spawns new `subprocess`es to execute the CLI tool I am shipping
- 5 unit tests to check the corner cases I have found

Are you ready?

`mutmut` found that I have at least 10 survived mutants (source code modifications):

```
» mutmut run

- Mutation testing starting -

These are the steps:
1. A full test suite run will be made to make sure we
   can run the tests successfully and we know how long
   it takes (to detect infinite loops for example)
2. Mutants will be generated and checked

Mutants are written to the cache in the .mutmut-cache
directory. Print found mutants with `mutmut results`.

Legend for output:
🎉 Killed mutants. The goal is for everything to end up in this bucket.
⏰ Timeout. Test suite took 10 times as long as the baseline so were killed.
🤔 Suspicious. Tests took a long time, but not long enough to be fatal.
🙁 Survived. This means your tests needs to be expanded.

1. Using cached time for baseline tests, to run baseline again delete the cache file

2. Checking mutants
⠇ 29/29  🎉 19  ⏰ 0  🤔 2  🙁 8%
```

## Deep dive into suspicious mutants

We can use `mutmut show` to show the diff for the survived mutants:

```
» mutmut show
To apply a mutant on disk:
    mutmut apply <id>

To show a mutant:
    mutmut show <id>


Suspicious 🤔 (2)

---- docker_image_size_limit.py (2) ----

17, 19

Survived 🙁 (8)

---- docker_image_size_limit.py (8) ----

2, 9, 10, 11, 20, 22, 27, 29
```

To view suspicious mutants we can use `mutmut show 17` and `mutmut show 19`:

```diff
» mutmut show 17
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -45,7 +45,7 @@
     try:
         image_limit = int(limit)
     except ValueError:
-        image_limit = parse_size(limit, binary=True)
+        image_limit = parse_size(limit, binary=False)

     if image_size > image_limit:
         return image_size - image_limit
```

and 

```diff
» mutmut show 19
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -47,7 +47,7 @@
     except ValueError:
         image_limit = parse_size(limit, binary=True)

-    if image_size > image_limit:
+    if image_size >= image_limit:
         return image_size - image_limit
     return 0
```

What these mutations are telling us? That our tests need more attention to corner cases.

## Things we did not test at all

### Mutant 2

There are several cases that survived our tests.

```diff
» mutmut show 2
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -8,9 +8,7 @@
 from docker import DockerClient, from_env
 from humanfriendly import format_size, parse_size

-_version = pkg_resources.get_distribution(
-    'docker_image_size_limit',
-).version
+_version = None


 def main():
```

And that's fine. We do not test that the version is correct. We can use `# pragma: no mutate` for this line.

### Mutant 9

We need to create a test case that will overflow our limit by just one byte.

```diff
» mutmut show 9
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -20,7 +20,7 @@
     oversize = check_image_size(client, arguments.image, arguments.size)

     exit_code = 0
-    if oversize > 0:
+    if oversize > 1:
         print('{0} exceeds {1} limit by {2}'.format(  # noqa: T001
             arguments.image,
             arguments.size,
```

Again, more attention to the corner cases.

### Mutant 10

See mutant #22

```diff
» mutmut show 10
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -21,7 +21,7 @@

     exit_code = 0
     if oversize > 0:
-        print('{0} exceeds {1} limit by {2}'.format(  # noqa: T001
+        print('XX{0} exceeds {1} limit by {2}XX'.format(  # noqa: T001
             arguments.image,
             arguments.size,
             format_size(oversize, binary=True),
```

Our output is mutated. And we missed that! 
For me, it is not a big problem. But for some apps it is absolutelly required test these kind of things.

### Mutant 11

To cover this case we need to assert that the correct number of overflow bytes is in our output.
Currently we do not do that.

```diff
» mutmut show 11
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -24,7 +24,7 @@
         print('{0} exceeds {1} limit by {2}'.format(  # noqa: T001
             arguments.image,
             arguments.size,
-            format_size(oversize, binary=True),
+            format_size(oversize, binary=False),
         ))
         exit_code = 1
     sys.exit(exit_code)
```

### Mutant 20

To cover this case we need to provide specific sizes. 
Which we do not do with our current testing strategy. 

```diff
» mutmut show 20
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -48,7 +48,7 @@
         image_limit = parse_size(limit, binary=True)

     if image_size > image_limit:
-        return image_size - image_limit
+        return image_size + image_limit
     return 0
```

### Mutant 22

The next three cases are the same: we mutate a strin, that is never checked.
To be sure that this won't happen for the strings you care about: 
write an integration test that literally checks the whole string output.

```diff
» mutmut show 22
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -54,7 +54,7 @@

 def _parse_args():
     parser = argparse.ArgumentParser(
-        description='Keep your docker images small',
+        description='XXKeep your docker images smallXX',
     )
     parser.add_argument(
         '--version', action='version', version=_version,
```

### Mutant 27

```diff
» mutmut show 27
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -60,7 +60,7 @@
         '--version', action='version', version=_version,
     )
     parser.add_argument(
-        'image', type=str, help='Docker image name to be checked',
+        'image', type=str, help='XXDocker image name to be checkedXX',
     )
     parser.add_argument(
         'size', type=str, help='Human-readable size limit: 102 MB, 1GB',
```

### Mutant 29

```diff
» mutmut show 29
--- docker_image_size_limit.py
+++ docker_image_size_limit.py
@@ -63,7 +63,7 @@
         'image', type=str, help='Docker image name to be checked',
     )
     parser.add_argument(
-        'size', type=str, help='Human-readable size limit: 102 MB, 1GB',
+        'size', type=str, help='XXHuman-readable size limit: 102 MB, 1GBXX',
     )
     return parser.parse_args()
```

## Conclusion

As you can see all the problems come from the exact one cause: 
we chose using real data without any mock or stubs.

And this data is unpredictable. We can not check that all corner cases work fine.
But, generally our app works correctly.

What to do to fix all these mutants? 
1. Use the combination of real data for E2E tests 
2. Use mocks / stubs to cover all corner cases you have